In [ ]:
# Cell 1 — Basic imports & device setup

import os
import sys
from pathlib import Path
from typing import List, Tuple

import torch
import torch.nn.functional as F
import numpy as np
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.cm as cm

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", device)

/home/jener05458/.conda/envs/EdgeMI/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Using device: cuda


In [ ]:
# Cell 2 — Load DeiT-base Teacher Model (same as Main Code)

try:
    from utils import build_model
except ImportError:
    # 프로젝트 구조가 다르면 여기에서 직접 경로 조정
    sys.path.append("../")   # 필요하면 수정
    from utils import build_model

def get_teacher_for_saliency(model_name: str):
    """
    model_name: 'deit_base_16_imagenet' 혹은 'deit_tiny_16_imagenet'
    """
    teacher_name_map = {
        'deit_tiny_16_imagenet': 'deit_tiny_patch16_224',
        'deit_base_16_imagenet': 'deit_base_patch16_224',
    }

    if model_name not in teacher_name_map:
        raise ValueError(f"Unknown model: {model_name}")

    arch_name = teacher_name_map[model_name]
    print(f"[INFO] Loading teacher model: {arch_name}")

    teacher = build_model(arch_name, Pretrained=True)
    teacher = teacher.to(device)
    teacher.eval()
    return teacher

TEACHER_MODEL_NAME = "deit_base_16_imagenet"   # 필요 시 tiny로 바꿔 가능
teacher = get_teacher_for_saliency(TEACHER_MODEL_NAME)

print("[INFO] Teacher model loaded and ready.")


[INFO] Loading teacher model: deit_base_patch16_224
[INFO] Teacher model loaded and ready.


In [ ]:
# Cell 3 — Load images from a folder (PNG/JPG)

from pathlib import Path
from typing import List, Tuple

def find_images_in_folder(folder: Path) -> List[Path]:
    """
    폴더 내부의 png/jpg/jpeg 이미지 경로를 전부 정렬하여 반환.
    """
    if not folder.exists():
        raise FileNotFoundError(f"[ERROR] 폴더가 존재하지 않습니다: {folder}")

    exts = ["*.png", "*.jpg", "*.jpeg"]
    paths = []

    for ext in exts:
        paths.extend(folder.glob(ext))

    paths = sorted(paths)
    if len(paths) == 0:
        raise FileNotFoundError(f"[ERROR] 이미지 파일을 찾지 못했습니다: {folder}")

    return paths


def load_images_for_saliency(folder: Path) -> List[Tuple[Path, torch.Tensor]]:
    """
    폴더 내부의 모든 이미지를 불러와
    (path, tensor[1,3,224,224]) 형태의 리스트로 반환.
    """
    image_paths = find_images_in_folder(folder)
    print(f"[INFO] Found {len(image_paths)} images in {folder}")

    to_tensor = transforms.ToTensor()
    loaded = []

    for p in image_paths:
        try:
            img = Image.open(p).convert("RGB")
            img_t = to_tensor(img)  # [3,H,W], float32
            img_t = img_t.unsqueeze(0).to(device)  # [1,3,H,W]
            loaded.append((p, img_t))
        except Exception as e:
            print(f"[WARN] Failed to load {p}: {e}")

    print(f"[INFO] Successfully loaded {len(loaded)} images.")
    return loaded


# --------------------------------------------------------
# 간단한 테스트 (원하는 폴더로 경로 수정해서 테스트하면 됨)
# --------------------------------------------------------

example_folder = Path("./dataset/deit_base_16_imagenet/DMI_iter/W4A8/DMI-4000-0-32-W4A8")
test_loaded = load_images_for_saliency(example_folder)
test_loaded[0][0], test_loaded[0][1].shape


[INFO] Found 32 images in dataset/deit_base_16_imagenet/DMI_iter/W4A8/DMI-4000-0-32-W4A8
[INFO] Successfully loaded 32 images.


(PosixPath('dataset/deit_base_16_imagenet/DMI_iter/W4A8/DMI-4000-0-32-W4A8/0-0.png'),
 torch.Size([1, 3, 224, 224]))

In [4]:
# Cell 4 — Saliency map computation (same as TBD-MI _saliency_p)

def compute_saliency_map(
    teacher,
    image_1chw: torch.Tensor,   # [1,3,224,224]
    target_class: int,
    patch_num: int = 197
):
    """
    TBD-MI의 saliency map 계산 방식 그대로 구현.
    입력 이미지는 반드시 [1,3,224,224] 이어야 함.
    """
    # ------------------------------
    # 1) prepare patch index
    # ------------------------------
    # DeiT의 full patch index: [0,1,2,...,196]
    current_abs_index = torch.arange(patch_num, device=device).unsqueeze(0)   # [1,197]
    next_relative_index = torch.arange(patch_num, device=device).unsqueeze(0) # [1,197]

    # ------------------------------
    # 2) grad를 받을 수 있도록 clone + requires_grad
    # ------------------------------
    x_req = image_1chw.detach().clone().requires_grad_(True)

    # ------------------------------
    # 3) forward + classification score 추출
    # ------------------------------
    logits, _, _ = teacher(x_req, current_abs_index, next_relative_index)

    # target class는 배치 1개 기준
    target_tensor = torch.tensor([[target_class]], device=device)

    score = logits.gather(1, target_tensor).sum()

    # ------------------------------
    # 4) gradient 계산
    # ------------------------------
    grad = torch.autograd.grad(
        score, x_req,
        create_graph=False,
        retain_graph=False
    )[0]    # [1,3,H,W]

    # ------------------------------
    # 5) saliency map 생성
    #    channel-wise max abs gradient
    # ------------------------------
    sal = grad.abs().amax(dim=1, keepdim=True)   # [1,1,H,W]

    # normalize (sum=1)
    sal_sum = sal.sum() + 1e-8
    sal_norm = sal / sal_sum

    return sal_norm  # [1,1,224,224]


In [8]:
# Cell 5 — Saliency heatmap + overlay visualization (fixed scaling)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

def save_saliency_visualization(
    image_1chw: torch.Tensor,      # [1,3,224,224]
    saliency_1chw: torch.Tensor,   # [1,1,224,224], sum=1 형태라고 가정
    save_path_base: Path,          # base path e.g. ".../img001"
    cmap_name: str = "jet",
    overlay_alpha: float = 0.45
):
    """
    image_1chw: [1,3,224,224], original image
    saliency_1chw: [1,1,224,224], normalized saliency map (sum=1)
    save_path_base: 기본 경로 (확장자 없이)
        -> save_path_base + "_heatmap.png"
        -> save_path_base + "_overlay.png"
        -> save_path_base + "_orig.png"
    """
    # ----------------------------------------
    # 1) 텐서를 numpy로 변환
    # ----------------------------------------
    img = image_1chw.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()  # [H,W,3]
    img = np.clip(img, 0, 1)

    sal = saliency_1chw.squeeze(0).squeeze(0).detach().cpu().numpy()     # [H,W]

    # 디버깅용: raw saliency 통계 출력
    print(f"[DEBUG] saliency raw min={sal.min():.3e}, max={sal.max():.3e}, mean={sal.mean():.3e}")

    # ----------------------------------------
    # 2) 시각화용으로 min-max 스케일링 (sum=1 → [0,1])
    # ----------------------------------------
    sal_vis = sal - sal.min()
    max_val = sal_vis.max()
    if max_val > 0:
        sal_vis = sal_vis / max_val
    else:
        # 완전 상수이면 그냥 0으로 두자
        sal_vis = np.zeros_like(sal_vis)

    # 디버깅용: scaled saliency 통계
    print(f"[DEBUG] saliency vis min={sal_vis.min():.3f}, max={sal_vis.max():.3f}")

    # ----------------------------------------
    # 3) heatmap 생성
    # ----------------------------------------
    cmap = cm.get_cmap(cmap_name)
    heatmap = cmap(sal_vis)[:, :, :3]  # RGB only (A 제거)

    heatmap_save_path = str(save_path_base) + "_heatmap.png"
    plt.imsave(heatmap_save_path, heatmap)
    print(f"[INFO] Saved heatmap: {heatmap_save_path}")

    # ----------------------------------------
    # 4) overlay 생성
    # ----------------------------------------
    overlay = img * (1 - overlay_alpha) + heatmap * overlay_alpha
    overlay = np.clip(overlay, 0, 1)

    overlay_save_path = str(save_path_base) + "_overlay.png"
    plt.imsave(overlay_save_path, overlay)
    print(f"[INFO] Saved overlay: {overlay_save_path}")

    # ----------------------------------------
    # 5) 원본 이미지도 같이 저장
    # ----------------------------------------
    orig_save_path = str(save_path_base) + "_orig.png"
    plt.imsave(orig_save_path, img)
    print(f"[INFO] Saved original: {orig_save_path}")


In [9]:
# Cell 6 — Full saliency pipeline: process every image in folder

from tqdm import tqdm

def process_folder_saliency(
    image_folder: Path,
    output_folder: Path,
    teacher,
    default_target_class: int = None,
    patch_num: int = 197,
):
    """
    폴더 내부의 모든 이미지를 saliency map으로 시각화하여 저장한다.
    default_target_class:
        - None  → 이미지마다 teacher prediction을 사용
        - int   → 모든 이미지에 동일한 target class 사용
    """
    output_folder.mkdir(parents=True, exist_ok=True)
    print(f"[INFO] Saving saliency results to: {output_folder}")

    # 1) 이미지 로딩
    loaded_images = load_images_for_saliency(image_folder)

    for idx, (img_path, img_tensor) in enumerate(tqdm(loaded_images, desc="Saliency")):
        # ------------------------------------------
        # 2) target class 결정
        # ------------------------------------------
        if default_target_class is None:
            # teacher prediction으로 자동 결정
            with torch.no_grad():
                logits, _, _ = teacher(
                    img_tensor,
                    torch.arange(patch_num, device=device).unsqueeze(0),
                    torch.arange(patch_num, device=device).unsqueeze(0),
                )
            target_class = int(logits.argmax(dim=1).item())
        else:
            target_class = default_target_class

        # ------------------------------------------
        # 3) saliency map 계산
        # ------------------------------------------
        saliency = compute_saliency_map(
            teacher,
            img_tensor,
            target_class=target_class,
            patch_num=patch_num,
        )

        # ------------------------------------------
        # 4) 저장 경로 구성
        # ------------------------------------------
        base_name = f"{idx:05d}"
        save_base = output_folder / base_name

        # ------------------------------------------
        # 5) 시각화 저장 (orig, heatmap, overlay)
        # ------------------------------------------
        save_saliency_visualization(
            img_tensor,
            saliency,
            save_base
        )

        # ------------------------------------------
        # 6) 콘솔 출력(optional)
        # ------------------------------------------
        print(f"[INFO] Processed: {img_path.name} → class {target_class}")

In [10]:
input_folder = Path("./dataset/deit_base_16_imagenet/DMI_iter/W4A8/DMI-4000-0-32-W4A8")
output_folder = Path("./observation/saliency/DMI_4000")

process_folder_saliency(
    image_folder=input_folder,
    output_folder=output_folder,
    teacher=teacher,
    default_target_class=None,   # automatic (teacher argmax)
    patch_num=197
)

[INFO] Saving saliency results to: observation/saliency/DMI_4000
[INFO] Found 32 images in dataset/deit_base_16_imagenet/DMI_iter/W4A8/DMI-4000-0-32-W4A8
[INFO] Successfully loaded 32 images.


Saliency:   0%|          | 0/32 [00:00<?, ?it/s]/tmp/ipykernel_1317435/2366521100.py:50: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)
Saliency:   3%|▎         | 1/32 [00:00<00:03,  9.19it/s]

[DEBUG] saliency raw min=1.457e-07, max=7.059e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00000_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00000_overlay.png
[INFO] Saved original: observation/saliency/DMI_4000/00000_orig.png
[INFO] Processed: 0-0.png → class 29
[DEBUG] saliency raw min=1.355e-07, max=3.048e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00001_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00001_overlay.png


Saliency:   9%|▉         | 3/32 [00:00<00:03,  8.85it/s]

[INFO] Saved original: observation/saliency/DMI_4000/00001_orig.png
[INFO] Processed: 0-1.png → class 96
[DEBUG] saliency raw min=2.009e-07, max=4.185e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00002_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00002_overlay.png
[INFO] Saved original: observation/saliency/DMI_4000/00002_orig.png
[INFO] Processed: 0-10.png → class 458
[DEBUG] saliency raw min=3.636e-08, max=3.631e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00003_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00003_overlay.png


...

[INFO] Saved original: observation/saliency/DMI_4000/00027_orig.png
[INFO] Processed: 0-5.png → class 350
[DEBUG] saliency raw min=2.157e-07, max=4.481e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00028_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00028_overlay.png
[INFO] Saved original: observation/saliency/DMI_4000/00028_orig.png
[INFO] Processed: 0-6.png → class 354
[DEBUG] saliency raw min=1.775e-07, max=3.846e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00029_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00029_overlay.png


Saliency:  97%|█████████▋| 31/32 [00:03<00:00,  8.99it/s]

[INFO] Saved original: observation/saliency/DMI_4000/00029_orig.png
[INFO] Processed: 0-7.png → class 360
[DEBUG] saliency raw min=9.906e-08, max=8.659e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00030_heatmap.png
[INFO] Saved overlay: observation/saliency/DMI_4000/00030_overlay.png
[INFO] Saved original: observation/saliency/DMI_4000/00030_orig.png
[INFO] Processed: 0-8.png → class 380
[DEBUG] saliency raw min=1.032e-07, max=5.084e-04, mean=1.993e-05
[DEBUG] saliency vis min=0.000, max=1.000
[INFO] Saved heatmap: observation/saliency/DMI_4000/00031_heatmap.png


Saliency: 100%|██████████| 32/32 [00:03<00:00,  9.11it/s]

[INFO] Saved overlay: observation/saliency/DMI_4000/00031_overlay.png
[INFO] Saved original: observation/saliency/DMI_4000/00031_orig.png
[INFO] Processed: 0-9.png → class 431
